In [1]:
import requests
import time
import pandas as pd
from api_keys import api_key

In [2]:
# Load the CSV file
file_path = 'Resourses/Fortune 500 Data.csv'  # Update this with the correct path to your CSV file
df = pd.read_csv(file_path)

# Assuming the tickers are in the second column (index 1)
tickers = df.iloc[:, 1].unique()  # Extract unique tickers to avoid duplicate API calls

# Limit to the first 100 tickers
tickers_100 = tickers[200:]
tickers

array(['WMT', 'AMZN', 'XOM', 'AAPL', 'UNH', 'CVS', 'BRKA', 'GOOGL', 'MCK',
       'CVX', 'ABC', 'COST', 'MSFT', 'CAH', 'CI', 'MPC', 'PSX', 'VLO',
       'F', 'HD', 'GM', 'ELV', 'JPM', 'KR', 'CNC', 'VZ', 'WBA', 'FNMA',
       'CMCSA', 'T', 'META', 'BAC', 'TGT', 'DELL', 'ADM', 'C', 'UPS',
       'PFE', 'LOW', 'JNJ', 'FDX', 'HUM', 'ET', 'FMCC', 'PEP', 'WFC',
       'DIS', 'COP', 'TSLA', 'PG', 'GE', 'ACI', 'MET', 'GS', 'SYY', 'RTX',
       'BA', 'SNEX', 'LMT', 'MS', 'INTC', 'HPQ', 'SNX', 'IBM', 'HCA',
       'PRU', 'CAT', 'MRK', 'INT', 'EPD', 'ABBV', 'PAGP', 'DOW', 'AIG',
       'AXP', 'CHTR', 'TSN', 'DE', 'CSCO', 'ALL', 'DAL', 'TJX', 'PGR',
       'AAL', 'PFGC', 'PBF', 'NKE', 'BBY', 'BMY', 'UAL', 'TMO', 'QCOM',
       'ABT', 'KO', 'ORCL', 'NUE', 'GD', 'COF', 'DINO', 'DG', 'ARW',
       'OXY', 'TRV', 'NOC', 'HON', 'MMM', 'USFD', 'WBD', 'LEN', 'DHI',
       'JBL', 'LNG', 'AVGO', 'KMX', 'SBUX', 'MOH', 'UBER', 'PM', 'NFLX',
       'NRG', 'MDLZ', 'DHR', 'CRM', 'PARA', 'CBRE', 'MU', 'V', 'SO',


In [6]:
# Manually add January 3, 2023
business_days_2023 = ['2023-01-03']

# Generate the first business day for each month from February to December 2023
additional_days = pd.date_range(start='2023-02-01', end='2023-12-31', freq='BMS')
additional_days = additional_days.strftime('%Y-%m-%d').tolist()

# Combine the lists
business_days_2023.extend(additional_days)

# Display the generated dates
print(business_days_2023)


['2023-01-03', '2023-02-01', '2023-03-01', '2023-04-03', '2023-05-01', '2023-06-01', '2023-07-03', '2023-08-01', '2023-09-01', '2023-10-02', '2023-11-01', '2023-12-01']


In [7]:
# Assuming tickers and business_days_2023 are already defined in your script

# Limit to the first 5 tickers
#first_100_tickers = tickers[:5]

# Limit to the first date
#business_days_2023 = business_days_2023[:2]
#business_days_2023

In [8]:
# Function to make the API call
def make_api_call(ticker, date, api_key):
    url = f"https://api.polygon.io/v1/open-close/{ticker}/{date}?adjusted=true&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        # Exclude 'status' and rename 'from' and 'symbol'
        return {
            "date": data["from"],
            "ticker": data["symbol"],
            "open": data["open"],
            "high": data["high"],
            "low": data["low"],
            "close": data["close"],
            "volume": data["volume"],
            "afterHours": data["afterHours"],
            "preMarket": data["preMarket"]
        }
    else:
        return None

def fetch_data_for_tickers(tickers, dates, api_key):
    # Initialize an empty DataFrame inside the function
    columns = ["date", "ticker", "open", "high", "low", "close", "volume", "afterHours", "preMarket"]
    df = pd.DataFrame(columns=columns)

    for ticker in tickers:
        for date in dates:
            result = make_api_call(ticker, date, api_key)
            if result:
                # Append the result to the DataFrame
                df = pd.concat([df, pd.DataFrame([result])], ignore_index=True)
            time.sleep(12)  # Sleep to limit to 5 calls per minute
    return df

# Call the function and store the results in the DataFrame
tickers_100 = fetch_data_for_tickers(tickers_100, business_days_2023, api_key)


In [17]:
data = pd.read_csv("Resourses/stockdata.csv")
data

,Index,date,ticker,open,high,low,close,volume,afterHours,preMarket
0,0,2023-01-03,WMT,142.55,143.71,142.0900,143.60,5749895.0,143.60,142.04
1,1,2023-02-01,WMT,143.66,145.61,142.7000,144.67,4283059.0,144.55,143.05
2,2,2023-03-01,WMT,141.06,141.28,139.3701,140.15,4726732.0,140.02,142.16
3,3,2023-04-03,WMT,147.73,148.95,147.5000,148.69,6588882.0,148.57,147.44
4,4,2023-05-01,WMT,151.04,152.54,150.9300,151.59,3803383.0,151.59,150.99
...,...,...,...,...,...,...,...,...,...,...
5640,5640,2023-08-01,RHI,73.72,74.05,72.8200,73.92,1350794.0,73.93,73.72
5641,5641,2023-09-01,RHI,74.80,75.29,74.3900,74.98,478574.0,74.98,73.97
5642,5642,2023-10-02,RHI,73.19,73.32,71.6900,72.03,920858.0,73.28,73.19
5643,5643,2023-11-01,RHI,74.33,74.74,73.3350,73.98,890369.0,73.98,74.33


In [10]:
combined_data = pd.concat([data, tickers_100], ignore_index=True)

In [16]:
combined_data.to_csv("Resourses/stockdata.csv", Index=False)

In [15]:
combined_data

,date,ticker,open,high,low,close,volume,afterHours,preMarket
0,2023-01-03,WMT,142.55,143.71,142.0900,143.60,5749895.0,143.60,142.04
1,2023-02-01,WMT,143.66,145.61,142.7000,144.67,4283059.0,144.55,143.05
2,2023-03-01,WMT,141.06,141.28,139.3701,140.15,4726732.0,140.02,142.16
3,2023-04-03,WMT,147.73,148.95,147.5000,148.69,6588882.0,148.57,147.44
4,2023-05-01,WMT,151.04,152.54,150.9300,151.59,3803383.0,151.59,150.99
...,...,...,...,...,...,...,...,...,...
5640,2023-08-01,RHI,73.72,74.05,72.8200,73.92,1350794.0,73.93,73.72
5641,2023-09-01,RHI,74.80,75.29,74.3900,74.98,478574.0,74.98,73.97
5642,2023-10-02,RHI,73.19,73.32,71.6900,72.03,920858.0,73.28,73.19
5643,2023-11-01,RHI,74.33,74.74,73.3350,73.98,890369.0,73.98,74.33
